In [25]:
import numpy as np 
import pandas as pd 
import json

In [26]:
movies = pd.read_csv("./tmdb_5000_movies.csv")
credits = pd.read_csv("./tmdb_5000_credits.csv")

df = movies.merge(credits,left_on='id',right_on='movie_id',how='left')

df = df[(df['status']=='Released')]

df = df[(df['vote_average']!=0)]

features_in_json = ['genres', 'keywords', 'production_companies', 'cast', 'crew', 'spoken_languages', 'production_countries']

In [27]:
df[df["id"] == 89861]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
3519,0,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,89861,"[{""id"": 131, ""name"": ""italy""}, {""id"": 8250, ""n...",en,Stiff Upper Lips,Stiff Upper Lips is a broad parody of British ...,0.356495,[],...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Stiff Upper Lips,10.0,1,89861,Stiff Upper Lips,"[{""cast_id"": 11, ""character"": ""Emily"", ""credit...","[{""credit_id"": ""52fe4a2f9251416c910c5edb"", ""de..."


In [28]:
for feature_name in features_in_json:
    print("Current:", feature_name)
    df[feature_name] = df[feature_name].apply(json.loads)
    
    if feature_name == 'crew':
        for index,i in zip(df.index,df[feature_name]):
            feature_list_1=[]
            limit = min(5, len(i))
            for j in range(limit):
                feature_list_1.append((i[j]['name']).strip())
            df.loc[index,feature_name]= str(feature_list_1)

    elif feature_name == 'cast':
        for index,i in zip(df.index,df[feature_name]):
            feature_list_1=[]
            limit = min(5, len(i))
            for j in range(limit):
                feature_list_1.append((i[j]['name']).strip())
            df.loc[index,feature_name]= str(feature_list_1)
    else:    
        for index,i in zip(df.index,df[feature_name]):
            feature_list_1=[]
            for j in range(len(i)):
                feature_list_1.append((i[j]['name']).strip())
            df.loc[index,feature_name]= str(feature_list_1)

    df[feature_name] = df[feature_name].str.strip('[]').str.replace(', ',',').str.replace("'",'').str.replace('"', '')
    df[feature_name] = df[feature_name].str.split(',')

Current: genres
Current: keywords
Current: production_companies
Current: cast
Current: crew
Current: spoken_languages
Current: production_countries


In [29]:
df[df["id"] == 89861]['cast'].iloc[0]

['Georgina Cates',
 'Peter Ustinov',
 'Prunella Scales',
 'Samuel West',
 'Sean Pertwee']

In [30]:
to_drop = []
for i in df.index:
    if (df['production_companies'][i] == [''] and df['cast'][i] == [''] and 
        df['crew'][i] == ['']):
        to_drop.append(i)
print('Dropping: ', str(len(to_drop)))
df = df.drop(to_drop, axis = 0)

Dropping:  7


In [31]:
features_keep = ['id', 'title', 'tagline', 'runtime', 'original_title', 'original_language', 'overview', 'keywords', 'genres', 'cast', 'crew', 'production_companies', 'production_countries', 'release_date', 'budget', 'revenue', 'vote_count', 'vote_average', 'popularity']

df['title'] = df['title_x']
df = df[features_keep]

In [32]:
df.dtypes

id                        int64
title                    object
tagline                  object
runtime                 float64
original_title           object
original_language        object
overview                 object
keywords                 object
genres                   object
cast                     object
crew                     object
production_companies     object
production_countries     object
release_date             object
budget                    int64
revenue                   int64
vote_count                int64
vote_average            float64
popularity              float64
dtype: object

In [33]:
df[df["id"] == 89861]

,id,title,tagline,runtime,original_title,original_language,overview,keywords,genres,cast,crew,production_companies,production_countries,release_date,budget,revenue,vote_count,vote_average,popularity
3519,89861,Stiff Upper Lips,NaN,99.0,Stiff Upper Lips,en,Stiff Upper Lips is a broad parody of British ...,"[italy, victorian england, young woman]",[Comedy],"[Georgina Cates, Peter Ustinov, Prunella Scale...","[Jeremy Bolt, Danny Hambrook, Gary Sinyor, Gar...",[],"[United Kingdom, India]",1998-06-12,0,0,1,10.0,0.356495


In [34]:
df['keywords'] = df['keywords'].apply(lambda kws: [kw.replace('"',"") for kw in kws])

In [35]:
df['keywords'] = df['keywords'].apply(lambda kws: [] if kws == [''] else kws)
df['genres'] = df['genres'].apply(lambda grs: [] if grs == [''] else grs)
df['cast'] = df['cast'].apply(lambda cst: [] if cst == [''] else cst)
df['crew'] = df['crew'].apply(lambda crw: [] if crw == [''] else crw)
df['production_companies'] = df['production_companies'].apply(lambda pcp: [] if pcp == [''] else pcp)
df['production_countries'] = df['production_countries'].apply(lambda pct: [] if pct == [''] else pct)

In [36]:
df.to_csv('tmdb_5000.csv', index=False)